## Rules
1. Four coloured team
2. Each team must be covered by 1 person everyday
* On weekdays, one employee can cover 2 teams
* On weekends, employees can only be on one team, but can cover 2 teams if in gold
* One cannot do both Red and Blue on weekends
* An employee cannot work for 3 weekends in a row
* Different days of the week have score. More the score, less ideal to work on:

| Saturday |Sunday | Tuesday |Thursday |Wednesday |Monday | Friday |
|----------|-------|---------|---------|----------|-------|--------|
| 5        |   4   |    3    |    2    |    2     |    1  |   1    |
        
* Design the algorithm to find the best schedule based on the lowest overall total
* Some employees will request days off, they should not be penaltized to have more work

In [1]:
import random
import calendar
import pandas as pd
import numpy as np
from numpy.random import choice
import timeit
import csv
import warnings
# from cProfile import profile
# , pstats, io
warnings.filterwarnings("ignore")
# import cProfile, pstats, io

# def profile(fnc):
    
#     def inner(*args, **kwargs):
#         pr = cProfile.Profile()
#         pr.enable()
#         retval=fnc(*args, **kwargs)
#         s = io.StringIO()
#         sortby = 'tottime'
#         ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
#         ps.print_stats()
#         print(s.getvalue())
#         return retval
    
#     return inner

#### Random year & month

In [2]:
# @profile
def get_dates():
    year = random.randint(1970,2100)
    month = random.randint(1,12)
#     print("Year :" , year," Month: ",month)

    #no of days
    start_day, no_of_days = calendar.monthrange(year, month)
    
    return year, month, no_of_days

#### Create month table

In [3]:
def get_score(weekday_name):
#     print(weekday_name)
    day_score = []
    for name in weekday_name:
        if name == 'Saturday' :
            day_score.append(5)
        elif name == 'Friday':
            day_score.append(4)
        elif name == 'Sunday':
            day_score.append(3)
        elif name == 'Tuesday' or name =='Thursday' :
            day_score.append(2)
        else:
            day_score.append(1)
    return day_score

def create_table(year, month, no_of_days):
    start_date = str(year) + '-' + str(month) +'-01'
    end_date = str(year) + '-' + str(month) +'-'+str(no_of_days)
    df = pd.DataFrame({'Date': pd.date_range(start_date, end_date)})
    df['Day'] = df.Date.dt.weekday_name
    df['Score'] = get_score(df.Date.dt.weekday_name)
    df['Blue'] = 'NA'
    df['Red'] = 'NA'
    df['Silver'] = 'NA'
    df['Gold'] = 'NA'
    return df

#### List of random no. of employees
#### Also select last 2 weekend's work randomly 

In [4]:
def create_employee_list():
    no_of_empl = random.randint(25,30)

    #6 random employees worked 2 weekends back
    weekend_1 = random.sample(range(1,no_of_empl+1), 6)
    weekend_2 = random.sample(range(1,no_of_empl+1), 6)

    empl_list = []
    for i in range(1,no_of_empl+1):
        one_weekend_back = 0
        two_weekend_back = 0
        if i in weekend_1:
            one_weekend_back = 1
        if i in weekend_2:
            two_weekend_back = 1
        if(i<10):
            empl_list.append(['emp_0' + str(i), 0, two_weekend_back, one_weekend_back, 0])
        else:
            empl_list.append(['emp_' + str(i), 0, two_weekend_back, one_weekend_back, 0])
            
#     Random no. of employee on holiday for random amount of days
    #5-10 employees taking a leave
    no_of_empl_leave = random.randint(5,10)

    #selecting random employees who are takeing a leave
    empl_list_leave = random.sample(range(0,no_of_empl+1), no_of_empl_leave)

    #assiging random amount of leave days (max 10)
    for loc,empl in enumerate(empl_list):
        if loc in empl_list_leave:
            empl[4] = random.randint(1,10)

    for num,empl in enumerate(empl_list):
        days_leave = random.sample(list(df_teams['Date']), empl[4])
        empl.append(days_leave)
        empl.append(10000)
        empl.append(no_of_days/(no_of_days-empl[4]))
    
# print("[Employee name, If working this week, If worked one weekend back, if worked two weekends back, No of days on leave, Weight, Score Multiplier]")
# for empl in empl_list:
#     print(empl)

    return empl_list, no_of_empl

In [5]:
def employee_schedule(year, month, empl_list):
    start_date = str(year) + '-' + str(month) +'-01'
    end_date = str(year) + '-' + str(month) +'-'+str(no_of_days)
    df = pd.DataFrame({'Date': pd.date_range(start_date, end_date)})
    for emp in empl_list:
        df[emp[0]] = 0
    df.loc['Total'] = 0.0001
    return df

#### Randomly assigning work to employees

In [6]:
def update_df_emp(teams, empl_choice, loc, score):
    
    for empl in empl_list:
        if empl == empl_choice:
            df_emp.loc[loc,empl[0]] += score*teams*empl[7]

def choose_empl(free_list, method):
    if method == 'random' :
        ## Select completly random employee for next day
        return random.choice(free_list)
    elif method == 'weighted' :
        ## Select an employee Psuedo-randomly (based on thier weights)
        totals = []
        running_total = 0

        for w in [empl[6] for empl in free_list]:
            running_total += w
            totals.append(running_total)

        rnd = random.random() * running_total
        for i, total in enumerate(totals):
            if rnd < total:
                return free_list[i]
    elif method == 'best' :
        ## Select employee who has lowest total score till date
        highest = -1
        best = []
        random.shuffle(free_list)

        for empl in free_list:
            if empl[6] > highest:
                highest = empl[6]
                best = empl

        return best
    else:
        return 0
    
def get_ml_inputs(score_next_day, free_list, is_weekend, day):
    random.shuffle(free_list)
    input_to_ml = []
    input_to_ml.append(score_next_day)
    for empl in free_list:
        input_to_ml.append(df_emp.loc['Total'][empl[0]])
    
    score_mults = []
    for empl in free_list:
        input_to_ml.append(empl[7])

    input_to_ml.append(is_weekend)
    
    with open('test.txt','w') as f:
        f.write(str(input_to_ml))
    
    np.save(f'selection/{day}.npy', input_to_ml)
    

In [7]:
# @profile
def assign_schedule(method):
    
#     df_teams = df_teams.sort_values(by=['Score'], ascending=False)

    for loc,date in enumerate(df_teams['Date']):

        score = df_teams.loc[loc,'Score']
        
        #weekend
        if(date.weekday_name == 'Saturday' or date.weekday_name == 'Sunday'):
            free_list = []
            for empl in empl_list:
                if ((empl[2] == 0 or empl[3] == 0) and empl[1] == 0) and (date not in empl[5]):
                    free_list.append(empl)
            
            #Inputs for ML
            get_ml_inputs(score, free_list, 1, date.day)

            gold_choice = choose_empl(free_list, method)
            df_teams.at[loc, 'Gold'] = gold_choice[0]

            free_list = [x for x in free_list if x != gold_choice]

            #assign same to one of the other three team (red,blue,silver)
            with_gold = choice(['Red','Blue','Silver'])
            df_teams.at[loc, with_gold] = gold_choice[0]
            update_df_emp(2, gold_choice, loc, score)

            #assign different employees to remaining 2 team
            other_choice_1 = choose_empl(free_list, method)
            free_list = [x for x in free_list if x != other_choice_1]
            other_choice_2 = choose_empl(free_list, method)

            if with_gold == 'Blue':
                df_teams.at[loc, 'Red'] = other_choice_1[0]
                df_teams.at[loc, 'Silver'] = other_choice_2[0]
                update_df_emp(1,other_choice_1,loc, score)
                update_df_emp(1,other_choice_2,loc, score)
            elif with_gold == 'Red':
                df_teams.at[loc, 'Blue'] = other_choice_1[0]
                df_teams.at[loc, 'Silver'] = other_choice_2[0]
                update_df_emp(1,other_choice_1,loc, score)
                update_df_emp(1,other_choice_2,loc, score)
            else:
                df_teams.at[loc, 'Blue'] = other_choice_1[0]
                df_teams.at[loc, 'Red'] = other_choice_2[0]
                update_df_emp(1,other_choice_1,loc, score)
                update_df_emp(1,other_choice_2,loc, score)

            #update weekend work status
            for i in range(len(empl_list)):
                if(empl_list[i]==gold_choice or empl_list[i]==other_choice_1 or empl_list[i]==other_choice_2):
                    empl_list[i][1] = 1

            if(date.weekday_name == 'Sunday'):
                for i in range(len(empl_list)):
                    empl_list[i][3] = empl_list[i][2]
                    empl_list[i][2] = empl_list[i][1]
                    empl_list[i][1] = 0
                    

    #   Weekday
        else:
            free_list = []
            for empl in empl_list:
                if date not in empl[5]:
                    free_list.append(empl)
            
            #Inputs for ML
            get_ml_inputs(score, free_list, 0, date.day)

            #Get random choice for Gold
            gold_choice = choose_empl(free_list, method)
            df_teams.at[loc, 'Gold'] = gold_choice[0]

            #assign same to one of the other three team (red,blue,silver)
            with_gold = choice(['Red','Blue','Silver'])
            df_teams.at[loc, with_gold] = gold_choice[0]

            update_df_emp(2, gold_choice, loc, score)
            free_list = [x for x in empl_list if x != gold_choice]

            #assign different employee to remaining 2 team
            other_choice = choose_empl(free_list, method)
            if with_gold == 'Blue':
                df_teams.at[loc, 'Red'] = other_choice[0]
                df_teams.at[loc, 'Silver'] = other_choice[0]
                update_df_emp(2,other_choice,loc, score)
            elif with_gold == 'Red':
                df_teams.at[loc, 'Blue'] = other_choice[0]
                df_teams.at[loc, 'Silver'] = other_choice[0]
                update_df_emp(2,other_choice,loc, score)
            else:
                df_teams.at[loc, 'Blue'] = other_choice[0]
                df_teams.at[loc, 'Red'] = other_choice[0]
                update_df_emp(2,other_choice,loc, score)

        df_emp.loc['Total'] = 0.0001
        df_emp.loc['Total'] = df_emp.sum()
        for empl in empl_list:
            empl[6] = 1 / df_emp.loc['Total'][empl[0]]
    
#     df_teams = df_teams.sort_values(by=['Date'])

#     return df_teams, df_emp, empl_list


In [8]:
for i in range(1):
    

    year, month, no_of_days = get_dates()

    df_teams = create_table(year, month, no_of_days)

    empl_list, no_of_empl = create_employee_list()

    df_emp = employee_schedule(year, month, empl_list)


#     if len(all_scores) !=31: pad zeros XXXX

#     [[next_day_score], 
#     [free->[current_score][score_mult]]

    method = 'random'
    assign_schedule(method)
    
    

In [9]:
df_emp.loc['Total']['emp_01']

10.333433333333334

In [10]:
workload_array = df_emp.loc['Total','emp_01':]
for num,workload in enumerate(workload_array):
    workload *= no_of_days / (no_of_days-empl_list[num][4])
    workload_array[num] = workload
wordload_std = np.std(workload_array)
std_no_of_empl = wordload_std * np.sqrt(no_of_empl)
score_to_minimize = wordload_std * np.sqrt(no_of_empl) / np.sqrt(no_of_days)

In [11]:
# tosave = [no_of_empl, no_of_days, wordload_std, std_no_of_empl, score_to_minimize]
# with open('std.csv','a') as f:
#     writer = csv.writer(f)
#     writer.writerow(tosave)

In [12]:
df_emp
score_to_minimize

7.82779165709884

In [13]:
df_teams

,Date,Day,Score,Blue,Red,Silver,Gold
0,2074-10-01,Monday,1,emp_04,emp_02,emp_02,emp_04
1,2074-10-02,Tuesday,2,emp_07,emp_22,emp_07,emp_22
2,2074-10-03,Wednesday,1,emp_28,emp_28,emp_18,emp_18
3,2074-10-04,Thursday,2,emp_14,emp_14,emp_07,emp_07
4,2074-10-05,Friday,4,emp_22,emp_17,emp_22,emp_17
5,2074-10-06,Saturday,5,emp_25,emp_23,emp_04,emp_23
6,2074-10-07,Sunday,3,emp_20,emp_21,emp_22,emp_20
7,2074-10-08,Monday,1,emp_23,emp_08,emp_23,emp_08
8,2074-10-09,Tuesday,2,emp_09,emp_09,emp_17,emp_17
9,2074-10-10,Wednesday,1,emp_02,emp_01,emp_02,emp_01
